In [1]:
# Created 2017, Zack Gainsforth
%pylab osx
import sys, os
import QuickPlot
import matplotlib.pylab as pylab
# pylab.rcParams['figure.figsize'] = 8, 6  # that's default image size for this interactive session
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
# from ipywidgets.widgets import interactive, fixed, interact
%config InlineBackend.figure_format = 'retina'

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, RobustScaler
from sklearn.decomposition import FastICA, PCA, FactorAnalysis, NMF

from tabulate import tabulate

Populating the interactive namespace from numpy and matplotlib


In [45]:
n_components=3

DecoMethods = [NMF, FastICA, PCA, FactorAnalysis]

#ICPNames = ['CrKa', 'CrKb', 'FeKa', 'FeKb', 'MgK', 'TiKa', 'VKa', 'OK']
ICPNames = ['Total counts', 'CrKa', 'CrKb', 'FeKa', 'FeKb', 'MgK', 'TiKa', 'VKa', 'OK']; ICPx=3; ICPy=3

ICPs = []
plt.figure('ICPs')
plt.subplot(ICPx, ICPy, 1)
for i, n in enumerate(ICPNames):
    ICPs.append(genfromtxt(n+'.txt'))
    plt.subplot(ICPx, ICPy, i+1)
    plt.imshow(ICPs[i], cmap='gray')
    plt.title(ICPNames[i])
   
    
data = np.zeros((len(ravel(ICPs[0])), len(ICPNames)))
for i in range(len(ICPNames)):
    data[:,i] = ICPs[i].ravel()
data.shape


(961, 9)

In [46]:

d = Normalizer().fit_transform(data)

figure('Decomposition Components')
plt.subplot(n_components,len(DecoMethods),1)
print(n_components)

for i, decomethod in enumerate(DecoMethods):
    if decomethod.__name__ == 'NMF':
        f = decomethod(n_components=n_components,alpha=0.8)
    elif decomethod.__name__ == 'PCA':
        f = decomethod(n_components=n_components)
    else:
        f = decomethod(n_components=n_components)
        
    fname = str(f.__class__).split('.')[-1].split("'")[0]
    
    dcomp = f.fit_transform(data)
    
    for n in range(n_components):
        plt.subplot(n_components, len(DecoMethods),i+1+n*len(DecoMethods))
        plt.imshow(dcomp[:,n].reshape((31,31)), cmap='gray')
        if n==0:
            plt.title(fname)
        #plt.subplot(n_components, len(DecoMethods),i+1+len(DecoMethods))
        #plt.imshow(dcomp[:,1].reshape((31,31)))
        plt.tight_layout()
        
    print(fname)
    Occupancy = f.components_
    Onorm = np.sum(Occupancy, axis=0)
    Occupancy /= Onorm
    #print(Occupancy)
    #rowids = ['Site 1', 'Site 2', 'Site 3']
    print(tabulate(Occupancy, headers=ICPNames, tablefmt='fancy_grid'))
#     print(f.components_)
#     for j, El in enumerate(ICPNames):
#         c0 = f.components_[0,j]
#         c1 = f.components_[1,j]
#         Occ = c0/(c0+c1)
#         print(El, ": ", Occ)
    print()

3
NMF
╒════════════════╤══════════╤══════════╤════════╤════════╤═══════╤═════════╤══════════╤═══════════╕
│   Total counts │     CrKa │     CrKb │   FeKa │   FeKb │   MgK │    TiKa │      VKa │        OK │
╞════════════════╪══════════╪══════════╪════════╪════════╪═══════╪═════════╪══════════╪═══════════╡
│      0.73068   │ 0.667902 │ 0.693871 │      1 │      1 │     1 │ 0.66522 │ 0.676927 │ 0.528682  │
├────────────────┼──────────┼──────────┼────────┼────────┼───────┼─────────┼──────────┼───────────┤
│      0.184693  │ 0.332098 │ 0.306129 │      0 │      0 │     0 │ 0.33478 │ 0.323073 │ 0.0220462 │
├────────────────┼──────────┼──────────┼────────┼────────┼───────┼─────────┼──────────┼───────────┤
│      0.0846278 │ 0        │ 0        │      0 │      0 │     0 │ 0       │ 0        │ 0.449272  │
╘════════════════╧══════════╧══════════╧════════╧════════╧═══════╧═════════╧══════════╧═══════════╛

FastICA
╒════════════════╤══════════╤═══════════╤═══════════╤═══════════╤════════════╤═══════

In [9]:
# NMF Matrix results
NewMat = np.array([[0.6666,0.3333,0], [1,0,0], [0.5287,0.0220,0.4493]]).T
print(NewMat)
MeasToOcc = np.linalg.inv(NewMat)
print(MeasToOcc)
print(np.dot(MeasToOcc, Occupancy[:,2]))

CrKa = 14750573
CrKb = 2045056
print(CrKb/(CrKa+CrKb))

[[ 0.6666  1.      0.5287]
 [ 0.3333  0.      0.022 ]
 [ 0.      0.      0.4493]]
[[ 0.          3.00030003 -0.14690986]
 [ 1.         -2.         -1.07878923]
 [ 0.          0.          2.2256844 ]]
[ 0.91847945  0.0816124   0.        ]
0.12176120346549689


In [14]:
# PCA Matrix results
NewMat = np.array([[0.242698,0.633069,0.124234], [-19.6978, 108.947, -88.2493], [-0.120269,0.204264,0.916004]]).T
print(NewMat)
MeasToOcc = np.linalg.inv(NewMat)
print(MeasToOcc)
print(np.dot(MeasToOcc, Occupancy[:,2]))



[[   0.242698  -19.6978     -0.120269]
 [   0.633069  108.947       0.204264]
 [   0.124234  -88.2493      0.916004]]
[[ 2.46155154  0.59870361  0.18968776]
 [-0.01158504  0.00495673 -0.00262641]
 [-1.44997157  0.39633995  0.81293901]]
[ 0.98548158 -0.00140888  0.01592619]


In [44]:
# FA Matrix results
NewMat = np.array([[1.0697,0.231393,-0.301091], [0.585155, -0.028183, 0.443028], [4.56256,-2.88621,-0.676348]]).T
print(NewMat)
MeasToOcc = np.linalg.inv(NewMat)
print(MeasToOcc)
print(np.dot(MeasToOcc, Occupancy[:,2]))




[[ 1.0697    0.585155  4.56256 ]
 [ 0.231393 -0.028183 -2.88621 ]
 [-0.301091  0.443028 -0.676348]]
[[ 0.53685728  0.99993055 -0.64547541]
 [ 0.4242433   0.26900351  1.71396218]
 [ 0.03889827 -0.26893552 -0.06848534]]
[ 0.82765593  0.17548678 -0.00314436]
